<center><h2><strong><font color="blue">WFH 2024 idBigData - FineTunning LLM</font></strong></h2></center>

<img alt="" src="https://github.com/taudataanalytics/WFH-idBigData-2024/blob/main/images/covers/cover_taudata_uin.jpg?raw=1"/>

# Jangan lupa mengganti Runtime menjadi GPU di Google Colab

In [1]:
import warnings; warnings.simplefilter('ignore')
from tqdm import tqdm
import os
os.environ["WANDB_MODE"] = "disabled"  # Disable wandb logging

try:
    import google.colab; IN_COLAB = True
    print("Installing the required modules")
    !pip install datasets --q
    print("preparing directories and assets")
    #!mkdir data images output models
    #!wget https://raw.githubusercontent.com/taudata...
except:
    IN_COLAB = False
    print("Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded")

import os
os.environ["WANDB_MODE"] = "disabled"  # Disable wandb logging
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Step 3: Load pre-trained DistilGPT2 and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set padding token to avoid padding error
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id  # Set pad token ID in model config

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Installing the required modules
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
preparing directories and assets


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

# ...

In [2]:
# Step 4: Load a small dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Select a small subset of the data for quick training
small_train_dataset = dataset["train"].select(range(100))  # Select first 100 examples
small_eval_dataset = dataset["validation"].select(range(10))  # Select first 10 examples

# Step 5: Tokenize the dataset
def tokenize_function(examples):
    # Tokenize and set 'labels' to 'input_ids' for supervised training
    tokens = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

# ...

In [3]:
# Step 6: Define training arguments (disable logging to any external services)
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    logging_strategy="no",  # Disable external logging
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,  # Adjust for more training
    weight_decay=0.01,
)

# Step 7: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)
"Done"

'Done'

<center><h2><strong><font color="blue">Train Model</font></strong></h2></center>

In [5]:
# Step 8: Train the model
trainer.train()

# Step 9: Save the fine-tuned model
model.save_pretrained("./fine-tuned-distilgpt2")
tokenizer.save_pretrained("./fine-tuned-distilgpt2")

# Step 10: Test the model with text generation
input_text = "Artificial intelligence is transforming"
inputs = tokenizer(input_text, return_tensors="pt", padding=True).to(device)
outputs = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],  # Pass attention mask explicitly
    max_length=50,
    num_return_sequences=1,
    pad_token_id=tokenizer.pad_token_id  # Set pad_token_id for reliable results
)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated text:", generated_text)

Epoch,Training Loss,Validation Loss
1,No log,1.283845
2,No log,1.286577
3,No log,1.287021


Generated text: Artificial intelligence is transforming the way we interact with the world and the world around us.


<center><h2><strong><font color="blue">Keterangan</font></strong></h2></center>

* **Dataset Loading**: We use a small subset of the WikiText dataset for simplicity. You can replace it with your own data if you wish.
* **Tokenization**: The tokenizer converts text into token IDs compatible with DistilGPT2.
* **Training**: The Trainer class handles training and evaluation with the specified parameters.
* **Testing**: After fine-tuning, the model is tested on a short input to demonstrate its ability to generate text.